## 🛫 External Stuff

### 🚗 DMVs

In [2]:
set nocount on; 
select * from sys.database_scoped_credentials;
select * from sys.external_data_sources;
select * from sys.external_file_formats;

Commands completed successfully.

Total execution time: 00:00:00.154

name,principal_id,credential_id,credential_identity,create_date,modify_date,target_type,target_id
AzureStorageCredential,1,65536,sqlva4onabrnvvfrkw,2019-09-20 21:03:49.030,2019-09-20 21:03:49.030,NULL,NULL


data_source_id,name,location,type_desc,type,resource_manager_location,credential_id,database_name,shard_map_name,connection_options,pushdown
65536,MyAzureStorage,wasbs://hiramblob@sqlva4onabrnvvfrkw.blob.core.windows.net/,HADOOP,0,NULL,65536,NULL,NULL,NULL,ON


file_format_id,name,format_type,field_terminator,string_delimiter,date_format,use_type_default,serde_method,row_terminator,encoding,data_compression,first_row
65539,parquet_file,PARQUET,NULL,,,0,NULL,NULL,NULL,org.apache.hadoop.io.compress.SnappyCodec,NULL
65542,csv2,DELIMITEDTEXT,",","""",,1,NULL,\n,UTF8,NULL,2
65543,csv,DELIMITEDTEXT,",","""",,1,NULL,\n,UTF8,NULL,1


## ⚓ Infrastructure

In [0]:
create database scoped credential hiramblob_cred
with 
     identity = 'sqlva4onabrnvvfrkw',
     secret = 'k7b/T5FY9nG77Bg10i9ATE3/+KBuOe6bpbbHTVj+tklh5d54HvqxKYO2Mxvr2bo98BHX8Hc36VUUVPnqUAdcLg==' --mykey
;

create external data source hiramblob_ds
with (    
    location = 'wasbs://hiramblob@sqlva4onabrnvvfrkw.blob.core.windows.net/',
    credential = hiramblob_cred,
    type = HADOOP
);

## 📝 Files

In [0]:
if exists (select 1 from sys.external_file_formats where name = 'parquet_ff') drop external file format parquet_ff;
create external file format parquet_ff
with (
    format_type = parquet,  
    data_compression = 'org.apache.hadoop.io.compress.SnappyCodec'  
);

if exists (select 1 from sys.external_file_formats where name = 'csv2_ff') drop external file format csv2_ff;
create external file format csv2_ff
with (
    format_type = delimitedtext,
    format_options(
        field_terminator = ',',
        string_delimiter = '"',
        first_row = 2, 
        use_type_default = true
    )
);

if exists (select 1 from sys.external_file_formats where name = 'csv_ff') drop external file format csv_ff;
create external file format csv_ff
with (
    format_type = delimitedtext,
    format_options(
        field_terminator = ',',
        string_delimiter = '"',
        use_type_default = true
    )
);

## 🎁 CETAS - Create External Table As Select - outbound

In [0]:
if exists (select * from sys.external_tables where name ='demoparquet')
begin 
    drop external table demoparquet;
end;
if exists (select * from sys.external_tables where name ='extBand')
begin 
    drop external table extBand;
end;
if exists (select * from sys.external_tables where name ='extBand2')
begin 
    drop external table extBand2;
end;

In [0]:
create external table demoparquet (
    BandId  int,
    Band    nvarchar(50),
    Album   nvarchar(50)
)
with (   
    location = 'demoparquet',  
    data_source = hiramblob_ds,  
    file_format = parquet_ff  
)
as 
    select * from dbo.Band;
go

create external table extBand
with (   
    location = 'extBand',  
    data_source = hiramblob_ds,  
    file_format = csv_ff  
)
as 
    select * from dbo.Band;
go

create external table extBand2
with (   
    location = 'extBand2',  
    data_source = hiramblob_ds,  
    file_format = csv2_ff  
)
as 
select * from dbo.Band;

In [0]:
exec sp_spaceused Band;
select count(*) from extBand;
select count(*) from extBand2;